# Notebook that calls COMSOL

# Imports

In [ ]:
import subprocess
import os
from time import sleep
import pickle

# Set the folder

In [ ]:
folder = "../COMSOL2python_interface/" # Set the folder to the COMSOL interface
assert os.path.isdir(folder), 'Set the correct path'

In [ ]:
comsol_file = "step2_flat_w_optimize.mph"
assert os.path.isfile(comsol_file), 'Set the correct path'

# Helper functions.

In [ ]:
def save_var(x, file):
    with open(file, 'wb') as f:
        pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

def load_var(file):
    with open(file, 'rb') as f:
        x = pickle.load(f)
    return x

In [ ]:
def clear(input=True, output=True):
    if input:
        COMSOL_inputs = [COMSOL_input for COMSOL_input in os.listdir(f"{folder}/COMSOL_input")]
        for COMSOL_input in COMSOL_inputs: os.remove(f"{folder}/COMSOL_input/" + COMSOL_input)
   
    if output:
        COMSOL_outputs = [COMSOL_output for COMSOL_output in os.listdir(f"{folder}/COMSOL_output")]
        for COMSOL_output in COMSOL_outputs:
            if COMSOL_output != 'running_instance':
                os.remove(f"{folder}/COMSOL_output/" + COMSOL_output)

In [ ]:
if not 'COMSOL' in subprocess.check_output("echo %PATH%".split(), shell=True).decode():
    raise Exception('COMSOL can not be called. '
                    'Move the whole folder to a machine where COMSOL can be called from cmd.')

In [ ]:
def run_as_cmd(cmd):
    assert isinstance(cmd, str)
    sleep(3)
    print('# ' + cmd)
    try:
        subprocess.call(cmd.split(),shell=True,stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [ ]:
def get_command():
    command  = "comsolbatch -inputfile " + comsol_file
    command += " -outputfile running_instances/" + comsol_file
    return(command)

In [ ]:
def run():
    run_as_cmd(get_command())
    sleep(3)

## Helper functions

In [ ]:
############################################################################
def set_output_ready():
    with open(folder + '_ready_output', 'w') as f:
        f.write(str(1))

############################################################################
def set_output_not_ready():
    if os.path.isfile(folder + '_ready_output'):
        os.remove(folder + '_ready_output')
        
############################################################################
def set_input_ready():
    with open(folder + '_ready_input', 'w') as f:
        f.write(str(1))

############################################################################
def set_input_not_ready():
    if os.path.isfile(folder + '_ready_input'):
        os.remove(folder + '_ready_input')
        
############################################################################
def is_input_ready():
    if '_ready_input' in os.listdir(folder):
        return True
    else:
        return False

# Reset und synchornize

In [ ]:
# Reset.
clear(input=False, output=True)
set_input_ready()

print('Waiting for other notebook to reset ... ', end='')
while (is_input_ready()):
    sleep(0.3)
    set_output_not_ready()
    
sleep(1)
set_output_not_ready()
        
print('Ready!')

# Compute Vext with COMSOL

In [ ]:
while (True):
    print('Waiting for input ...', end=' ')
    while (not is_input_ready()):
        sleep(0.3)
    print('Computing output ... ')
    clear(input=False, output=True)
    set_input_not_ready()
    run()
    set_output_ready()
    print('Done!')